In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import random
import matplotlib.image as mpimg
import splitfolders
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
from keras.models import Sequential
from keras import models
from keras.layers import Dense, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
import shutil
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from IPython.display import SVG
from scipy import ndimage
import pathlib as Path

In [2]:
input_data_path =r'C:/Users/conm/Desktop/Stenosis-Project/Stenosis detection/input'

train_path = r'C:\Users\conm\Desktop\Stenosis-Project\output\train\stenosis'

train_path1 = r'C:\Users\conm\Desktop\Stenosis-Project\train_images'

test_path = r'C:\Users\conm\Desktop\Stenosis-Project\test_images'

val_path = r'C:\Users\conm\Desktop\Stenosis-Project\output\val\stenosis'

In [3]:
test_labels_csv = pd.read_csv('test_labels.csv')
test_labels_csv

,filename,width,height,class,xmin,ymin,xmax,ymax
0,14_029_7_0058.bmp,800,800,Stenosis,288,369,328,398
1,14_024_1_0052.bmp,800,800,Stenosis,166,160,202,202
2,14_095_2_0075.bmp,800,800,Stenosis,461,337,492,379
3,14_075_6_0035.bmp,512,512,Stenosis,128,136,189,158
4,14_081_7_0026.bmp,800,800,Stenosis,323,177,351,230
...,...,...,...,...,...,...,...,...
828,14_066_8_0028.bmp,800,800,Stenosis,125,175,161,216
829,14_048_1_0057.bmp,1000,1000,Stenosis,264,427,306,458
830,14_051_2_0051.bmp,512,512,Stenosis,82,297,110,344
831,14_051_2_0025.bmp,512,512,Stenosis,77,248,102,284


In [4]:
train_labels_csv = pd.read_csv('train_labels.csv')
train_labels_csv

,filename,width,height,class,xmin,ymin,xmax,ymax
0,14_024_2_0042.bmp,800,800,Stenosis,235,156,282,192
1,14_031_4_0031.bmp,800,800,Stenosis,419,319,446,349
2,14_046_5_0066.bmp,1000,1000,Stenosis,679,307,699,336
3,14_010_6_0035.bmp,512,512,Stenosis,94,244,126,286
4,14_014_1_0015.bmp,512,512,Stenosis,163,118,203,151
...,...,...,...,...,...,...,...,...
6737,14_066_7_0059.bmp,800,800,Stenosis,229,100,254,160
6738,14_088_8_0077.bmp,800,800,Stenosis,512,319,576,347
6739,14_095_7_0045.bmp,800,800,Stenosis,199,218,229,273
6740,14_051_6_0026.bmp,512,512,Stenosis,258,181,277,216


In [5]:
val_labels_csv = pd.read_csv('val_labels.csv')
val_labels_csv

,filename,width,height,class,xmin,ymin,xmax,ymax
0,14_029_5_0059.bmp,800,800,Stenosis,192,423,237,459
1,14_006_1_0138.bmp,800,800,Stenosis,471,438,560,575
2,14_087_1_0090.bmp,800,800,Stenosis,256,250,335,300
3,14_021_37_0019.bmp,512,512,Stenosis,187,408,239,431
4,14_057_3_0061.bmp,1000,1000,Stenosis,389,117,443,174
...,...,...,...,...,...,...,...,...
745,14_029_1_0068.bmp,800,800,Stenosis,297,265,356,310
746,14_095_2_0035.bmp,800,800,Stenosis,435,375,473,416
747,14_095_1_0080.bmp,800,800,Stenosis,501,498,546,529
748,14_048_3_0074.bmp,1000,1000,Stenosis,458,181,479,199


In [18]:
train_images = []
train_targets = []
train_labels = []
classes = ['Stenosis']

In [20]:
for index, row in train_labels_csv.iterrows():
    
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row
    
    train_image_fullpath = os.path.join(train_path, filename)
    train_img = keras.preprocessing.image.load_img(train_image_fullpath, target_size=(256, 256))
    train_img_arr = keras.preprocessing.image.img_to_array(train_img)
    
    # xmin = xmin / width
    # ymin = ymin / height
    # xmax = xmax / width
    # ymax = ymax / height
    xmin = round(xmin/ width, 2)
    ymin = round(ymin/ height, 2)
    xmax = round(xmax/ width, 2)
    ymax = round(ymax/ height, 2)
    
    train_images.append(train_img_arr)
    train_targets.append((xmin, ymin, xmax, ymax))
    train_labels.append(class_name)

In [13]:
val_images = []
val_targets = []
val_labels = []
classes = ['Stenosis']

In [14]:
for index, row in val_labels_csv.iterrows():
    
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row
    
    val_image_fullpath = os.path.join(val_path, filename)
    val_img = keras.preprocessing.image.load_img(val_image_fullpath, target_size=(256, 256))
    val_img_arr = keras.preprocessing.image.img_to_array(val_img)
    
    # Normalise the xmin, ymin, xmax, ymax values
    # xmin = xmin / width
    # ymin = ymin / height
    # xmax = xmax / width
    # ymax = ymax / height
    xmin = round(xmin/ width, 2)
    ymin = round(ymin/ height, 2)
    xmax = round(xmax/ width, 2)
    ymax = round(ymax/ height, 2)
    
    val_images.append(val_img_arr)
    val_targets.append((xmin, ymin, xmax, ymax))
    val_labels.append(class_name)

In [29]:
# Create YOLOv4 model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(4, activation='sigmoid'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_30 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 26, 26, 128)     

In [34]:
# shape of training_data_resized
train_images = np.array(train_images, dtype=np.float16)

In [16]:
val_images = np.array(val_images, dtype=np.float16)

In [17]:
val_images.shape

(750, 256, 256, 3)

In [30]:
val_targets = np.array(val_targets, dtype=np.float16)

In [31]:
train_targets = np.array(train_targets, dtype=np.float16)

In [32]:
train_targets.shape

(6742, 4)

In [36]:
# Compile the model with loss mse
model.compile(loss='mse', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Train the model
model.fit(train_images, train_targets, validation_data=(val_images, val_targets), epochs=10, batch_size=128, verbose=1)

Epoch 1/10
53/53 [==============================] - 240s 4s/step - loss: 0.0654 - accuracy: 0.4898 - val_loss: 0.0179 - val_accuracy: 0.7773
Epoch 2/10
 6/53 [==>...........................] - ETA: 3:25 - loss: 0.0177 - accuracy: 0.7083

KeyboardInterrupt: 